In [ ]:
import uproot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

import sys

# import plothist
# plt.style.use("belle2")
# from plothist import make_2d_hist, plot_2d_hist_with_projections

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/07-Python_Functions/")

### Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal","All","BB","ccbar","ddbar","ssbar","taupair","uubar","uds"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/home/belle2/amubarak/TopoAna/Background/Completed_TopoAna/TopoAna_"+ s +".root:Dstree",library='pd')

In [ ]:
pd.set_option('display.max_rows', 200000)
pd.set_option('display.max_columns', 200000)

In [ ]:
DataFrames["ccbar"].isna().sum()

The line below is to look at the available variables.

In [ ]:
DataFrames["Signal"].columns.tolist()

The line below only looks at the interesting variables.

In [ ]:
features = DataFrames["All"].columns.tolist()
features.remove('Ds_goodDsplus')

In [ ]:
# Suffix removal from String list
# using list comprehension + endswith()
features = [ele for ele in features if not ele.endswith('charge')
            and not ele.endswith('trackTime')
            and not ele.endswith('isSignal') and not ele.endswith('PDG')
            and not ele.endswith('mcErrors')
            and not ele.endswith('genMotherPDG') and not ele.endswith('genMotherPDG_1') and not ele.endswith('genMotherPDG_2')
            and not ele.endswith('genMotherID') and not ele.endswith('genMotherID_1') and not ele.endswith('genMotherID_2')
            and not ele.endswith('genParticleID')
            and not ele.endswith('nMCDaughters') and not ele.endswith('nMCMatches') and not ele.endswith('mcMatchWeight')
            and not ele.endswith('mcInitial') and not ele.endswith('mcVirtual')
            and not ele.endswith('mcSecPhysProc') and not ele.endswith('mcPrimary') and not ele.endswith('mcPhotos') 
            and not ele.endswith('isMisidentified') and not ele.endswith('isCloneTrack') and not ele.endswith('isOrHasCloneTrack')
            and not ele.endswith('mcP') and not ele.endswith('mcE')
            and not ele.endswith('seenInCDC') and not ele.endswith('seenInPXD') and not ele.endswith('seenInSVD')
            ]

# Initializing remove list
remove_list = ['__experiment__','__run__','__event__','__production__','__candidate__','__ncandidates__','__weight__','__eventType__',
               'e_pminusptrue_p','e_pminusptrue_old','e_mcMother_E','e_mcMother_p','e_mcMother_mcDecayVertexFromIPDistance',
               'e_Motherptrueminusp_p','e_Motherptrueminusp_E',
               'e_M','K_M','pi_M',
               'D0_mcMother_mcDaughter_1_pt','D0_mcMother_mcDaughter_1_pz','D0_mcMother_mcDaughter_1_cos_theta','D0_ifNANgiveX_isSignal_5',
               'Ds_M_pi','Ds_M_uncorrected','Ds_daughterCombination_M_0_0_1',
               'Ds_decayAngle_0','Ds_cos_decayAngle_0','Ds_decayAngle_1','Ds_cos_decayAngle_1',
               'Ds_openangle','Ds_Angle_D0e',
               'Ds_genNStepsToDaughter_0','Ds_genNStepsToDaughter_1','Ds_genNMissingDaughter_11','Ds_genNMissingDaughter_22',
               'Ds_ifNANgiveX_isSignal_5',
               'Ds_Comb','Ds_Failed','Ds_Dstar0','Ds_Dstarplus','Ds_Other','Ds_Signal',
               'Ds_gammaveto_em_GoodElectron',
               'Ds_D0_Dstarplus','Ds_D0_Dstar0','Ds_D0_NoDstarplusDstar0','Ds_D0_Other',
               'Ds_azimuthalAngleInDecayPlane_0_1',
               'Ds_daughterMotherDiffOf_0_vertexDistance',
               'Ds_gammaveto_ep_M','Ds_gammaveto_em_M','Ds_Dstar0Mode1_chiProb','Ds_Dstar0Mode2_chiProb',
               'Ds_D0_sideband',
               'IPX','IPY','IPZ']
 
# Removing elements present in other list
# using list comprehension
features = [i for i in features if i not in remove_list]

In [ ]:
features

### Setup
The code below will be used to apply cuts to the data.  
The range of the plots.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Fake D0 Suppression
#------------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_extraInfo_FastBDT']>=0.58]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_extraInfo_FastBDT']>=0.58]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_extraInfo_FastBDT']>=0.58]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_gammaveto_M_Correction']>=0.1]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb_Ds_rank']==1]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['Ds_chiProb_Ds_rank']==1]
# KaonID
#-----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['K_kaonID']>=0.5]
# DataFrames["All"] = DataFrames["All"][DataFrames["All"]['K_kaonID']>=0.5]

# Variable Comparison
Below, the for loop attempts to go through the different variable to see which holds the most discriminating power.

In [ ]:
# Stacked = False
# Density = True
# Bins = 50
# Signal = True

# Op = 0.58
# dM = 0.1
# Hits = -1

# # label1= 'failed'
# # label2= r'$e^{+}\;\mathrm{from}\;\gamma$'
# # label3= r'$e^{+}\;\mathrm{from}\;\pi^{0}$'
# # label4= 'misID'

# for i in features:
#       if Signal == False:

#             label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
#             label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
#             label3= r'$D^{0}$'
#             label4= r'$Other$'

#             labels=[label1,label2,label3,label4]
#             colors=['C2','C1','C4','C5']
#             data=[
#                 DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
#                 DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
#                 DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
#                 DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
#                 ]
            
#             Range = [DataFrames['Signal'][i].min(axis=0),DataFrames['Signal'][i].max(axis=0)]
            
#             factor = 0.25
#             plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op)][i]), ls='--', linewidth=1.5)
#             plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

#             plt.ylabel('Entries')
#             plt.xlabel(i)
#             plt.legend()
#             plt.show()
#       else:
# #             label3= r'$fail$'
# #             label3= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

#             labels=['Background']
# #             colors=['C4']
#             data=[
#                 DataFrames["All"][(DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)][i],
# #                 DataFrames["All"][(abs(DataFrames["All"]["Ds_mcPDG"])==413) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['e_nPXDHits']>Hits) & (DataFrames["All"]["Ds_extraInfo_FastBDT"]>=Op)][i],
#                 DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op)][i]
#             ]
# #             data=[DataFrames["All"][i]]
#             if i == 'e_chi2':
#                 Range = [0,200]
#             else:
#                 # Range = [-0.25,0.25]
#                 Range = [data[0].min(axis=0),data[0].max(axis=0)]
# #             z_scores_0 = stats.zscore(data[0])
# #             z_scores_1 = stats.zscore(data[1])
# #             data[0] = data[0][(z_scores_0 > -3) & (z_scores_0 < 3)]
# #             data[1] = data[1][(z_scores_1 > -3) & (z_scores_1 < 3)]
# #             Range = [0,0.5]
# #             Range = [-0.025,0.025]
            
            
#             plt.hist(data[1], label=r'$Signal$', density=Density, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
#             plt.hist(data[0:1], label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)

#             plt.ylabel('Entries')
#             plt.xlabel(i)
# #             plt.yscale("log")
#             plt.legend()
#             plt.show()

# Single Variable  
The code below is to look at any specific variable closely.

In [ ]:
features

In [ ]:
Stacked = False
Density = True
Bins = 50
Range = [0, 0.5]
Op = -1
dM = -1
# i = 'Ds_extraInfo_FastBDT'
# i = 'Ds_chiProb'
# i = 'Ds_Ds_starminusDs_M_Correction'
# i = 'Ds_KaonCount_3'
# i = 'Ds_chiProb'
# var = "Ds_L_diff"
i = "Ds_gammaveto_M_Correction"
# var = 'e_cos_theta'
# var = 'e_pt'
perBin = ((Range[1] - Range[0])/Bins)*1000
# perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Signal$'
label2= r'$Background$'

labels=[label1,label2]
colors=["#1f77b4","#d62728"]

data = [
        DataFrames["Signal"][i], # (DataFrames["Signal"]['Ds_charge']==-1) & 
        DataFrames["All"][i]
       ]


plt.hist(data, color=colors, label=labels, alpha=1, range=Range, stacked=Stacked, density=Density, bins=Bins, histtype='step')
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.58,ls='--',color='gray')

# Title
#---------
# plt.title(r'$Signal: Charge(D_s^{+})=Positive$', loc = "left")
# plt.title(r'$\bf Signal\;Events$', loc = "right")
# plt.title(r'$\int\mathcal{L}dt\approx\;100$ fb$^{-1}$', loc = "left")
# plt.title(r'$\bf Generic\;c\bar{c}\;Events$', loc = "right")
# Label
#---------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
# plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# plt.ylabel(r'$Entries$')
# plt.xlabel(r'$n_{K^{-}}(ROE)$')
# plt.xlabel(r'$n_{K^{-}} - n_{K^{+}} + n_{K_{s}^{0}} \; (ROE)$')
# plt.yscale("log") 
# plt.xscale("log") 
plt.legend()
plt.show()

In [ ]:
Stacked = False
Density = True
Bins = 50
Range = [0, 0.4]
Op = -1
dM = -1
# i = 'Ds_chiProb'
# i = 'Ds_extraInfo_FastBDT'
# i = 'D0_chiProb'
i = 'Ds_Ds_starminusDs_M_Correction'
# i = "Ds_gammaveto_M_Correction"
# i = 'D0_chiProb'
# i = "Ds_L_diff"
# var = 'e_cos_theta'
# i = 'e_pt'
perBin = ((Range[1] - Range[0])/Bins)*1000
# perBin = ((Range[1] - Range[0])/Bins)
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'
label2= r'$D^{*0} \rightarrow D^{0} \gamma, D^{0} \pi^{0}$'
label3= r'$D^{0}$'
label4= r'$Other$'

labels=[label1,label2,label3,label4]
colors=['C2','C1','C4','C5']
data=[
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstarplus'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Dstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_NoDstarplusDstar0'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
      DataFrames["All"][(abs(DataFrames["All"]['Ds_D0_Other'])==1) & (DataFrames["All"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["All"]['Ds_extraInfo_FastBDT']>=Op)][i],
      ]

factor = 0.5
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]["Ds_extraInfo_FastBDT"]>=Op)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.58,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BCS: Vertex \; Fit \; \chi^{2}$', loc = "left")
# Label
#-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.xlabel(r'$p-value_{IP}(D_{s}^{+})$')
# plt.xlabel(r'$Fake D^{0} Suppression(D^{0})$')
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c)$'.format(width = perBin))
# plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.xlabel(r'$p_{t} \; (e^{+})\;[GeV/c]$')
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.xlabel(r'$\Delta m(D_s^{*+} - D_{s}^{+})\;[GeV/c^{2}]$')
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.xlabel(r'$cos\theta \; (e^{+})$')
# plt.xlabel(r'$p-value(D^{0})$')
# plt.ylabel(r'$Entries/(\; {width:.2f}\;)$'.format(width = perBin))
# plt.xlabel(r'$\mid \vec{x}_{D_{s}^{+}} - \vec{x}_{D^{0}} \mid \; [cm]$')
# plt.xlabel(r'$dz \; (e^{+}) \; [cm]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 12000)
plt.legend()
plt.show()

In [ ]:
#  'Ds_nROE_KS0',
#  'Ds_nROE_Charged_roe_mask_211',
#  'Ds_nROE_ParticlesInList_pi_loose_roe_mask',
#  'Ds_nROE_Charged_roe_mask_321',
#  'Ds_nROE_ParticlesInList_K_loose_roe_mask',
#  'Ds_nROE_ParticlesInList_K_plus_roe_mask',
#  'Ds_nROE_ParticlesInList_K_minus_roe_mask',

In [ ]:
DataFrames["ccbar"][['__event__','Ds_nROE_KS0','Ds_nROE_ParticlesInList_K_loose_roe_mask','Ds_nROE_ParticlesInList_K_plus_roe_mask','Ds_nROE_ParticlesInList_K_minus_roe_mask']].head(30)

In [ ]:
# GoodConversionVariablesVeto = ['e_d0',
#                                'e_dcosTheta'
#                                'e_abs_dr',
#                                'Ds_useRestFrame_daughterDiffOf_0_1_p',
#                                'Ds_useRestFrame_daughterAngle_0_1',
#                                'Ds_azimuthalAngleInDecayPlane_0_1'
#                                'Ds_useRestFrame_daughterMotherDiffOf_0_p', # There are two unique areas
#                                'Ds_flightDistance' # The signal is slightly shifted upwards
#                                'Ds_daughterDiffOf_0_1_x', #Slight Shift
#                                'Ds_daughterDiffOf_0_1_y', #Slight Shift
#                                'Ds_daughterDiffOf_0_1_distance', #Slight Shift
#                                'Ds_x', # Clear Shift
#                                'Ds_y', # Clear Shift
#                               ]

# Fit Variable

In [ ]:
Stacked = False
Density = False
Bins = 50
# i = 'Ds_diff_D0pi'
i = "Ds_gammaveto_M_Correction"
# i = 'Ds_massDifference_0'
Range = [-0.05, 1]
dM = -1
Hits = -1
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$Comb.$'
label3= r'$NaN$'
label4= r'$D^{*0}$'
label5= r'$D^{*+} \rightarrow D^{0} \pi^{+}$'

labels=[label1,label2,label3,label4,label5]
colors=['C2','C1','C3','C4','C5']
data=[DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_Other"])==1) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==23) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["ccbar"][(DataFrames["ccbar"]["Ds_mcPDG"].isna()) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==423) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i],
      DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==413) & (DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=dM)][i]
      ]

factor = 0.5
plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM) & (DataFrames["Signal"]['e_nPXDHits']>Hits)][i], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=dM)][i]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(Range[0],0.15,color='gray',alpha=0.2)
# plt.axvline(0.15,ls='--',color='gray')

# Title
#--------
# plt.title(r'$BCS: Vertex \; Fit \; \chi^{2}$', loc = "left")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 2000)
plt.legend()
plt.show()

# 2-D Histogram

In [ ]:
# # name_x = 'Ds_massDifference_0'
# name_x = 'Ds_dM_D0pi'
# name_y = 'Ds_Angle_D0e'
# Sample = "ccbar"

# h = make_2d_hist([DataFrames[Sample][name_x], DataFrames[Sample][name_y]], bins=[50, 50], range=[[0.135,0.4],[0,3.14]])

# (
#     fig,
#     ax_2d,
#     ax_x_projection,
#     ax_y_projection,
#     ax_colorbar,
# ) = plot_2d_hist_with_projections(
#     h,
#     xlabel=r'$\Delta m\;[GeV/c^{2}]$',
#     ylabel=r'$\theta(D^{0}e^{+})\;[rad]$',
#     ylabel_x_projection="Entries",
#     xlabel_y_projection="Entries",
#     offset_x_labels=True,
#     colorbar_kwargs={"label": "Entries"},
#     figsize=(6,6)
# )

In [ ]:
print(abs(DataFrames["ccbar"][(abs(DataFrames["ccbar"]["Ds_mcPDG"])==10411)]['Ds_genMotherPDG']).value_counts(dropna=False).head(100))